In [1]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import pickle
import pandas as pd
import numpy as np


In [78]:
model = load_model('model.h5')

with open('label_encoder_gender.pkl' , 'rb') as file : 
    label_encoder_gender = pickle.load(file)


with open('onehot_encoder_geo.pkl' , 'rb') as file : 
    label_encoder_geo = pickle.load(file)


with open('scaler.pkl' , 'rb') as file : 
    scaler = pickle.load(file)




In [32]:
def generate_samples(n=5, seed=None):
    if seed is not None:
        np.random.seed(seed)
    countries = ['France', 'Spain', 'Germany']
    genders = ['Male', 'Female']
    samples = []
    for _ in range(n):
        samples.append({
            "CreditScore": int(np.random.randint(350, 851)),
            "Geography": np.random.choice(countries),
            "Gender": np.random.choice(genders),
            "Age": int(np.random.randint(18, 92)),
            "Tenure": int(np.random.randint(0, 11)),
            "Balance": float(np.round(np.random.uniform(0, 250000), 2)),
            "NumOfProducts": int(np.random.randint(1, 5)),
            "HasCrCard": int(np.random.choice([0, 1])),
            "IsActiveMember": int(np.random.choice([0, 1])),
            "EstimatedSalary": float(np.round(np.random.uniform(1000, 200000), 2))
        })
    
    if n == 1:
        return samples[0]  # Return single dict object for one sample
    else:
        return samples    # Return list of dict objects for multiple samples

In [102]:
input_data = generate_samples(1)
input_data

{'CreditScore': 536,
 'Geography': 'Spain',
 'Gender': 'Female',
 'Age': 20,
 'Tenure': 3,
 'Balance': 81489.73,
 'NumOfProducts': 4,
 'HasCrCard': 0,
 'IsActiveMember': 1,
 'EstimatedSalary': 102906.52}

In [103]:

geo_encoded = label_encoder_geo.transform([[input_data['Geography']]]).toarray()
geo_encoded_df = pd.DataFrame(geo_encoded,columns=label_encoder_geo.get_feature_names_out(['Geography']))


NotFittedError: This OneHotEncoder instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.

In [109]:

input_df = pd.DataFrame([input_data])
input_df['Gender']=label_encoder_gender.transform(input_df['Gender'])
input_df  = pd.concat([input_df.drop('Geography' , axis=1) , geo_encoded_df],axis=1)

In [95]:
input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,402,0,87,6,168033.89,3,1,0,49189.83,0.0,0.0,1.0


In [98]:
input_scaled=scaler.transform(input_df)
input_scaled

array([[-2.59077591, -1.09499335,  4.5731981 ,  0.34567966,  1.47191829,
         2.53355998,  0.64920267, -1.02583358, -0.89091837, -0.99850112,
        -0.57946723,  1.73494238]])

In [108]:
predeict = model.predict(input_scaled)

if predeict[0][0] > 0.5 :
    print('fucked off')
else :
    print('Yess you are in ')

1/1 [==============================] - 0s 12ms/step
Yess you are in 


Testing with 3 random samples:



NotFittedError: This OneHotEncoder instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.